In [1]:
# 🏠 House Prices - XGBoost
# =====================================================
import numpy as np
import pandas as pd
from pathlib import Path
# XGBoost
from xgboost import XGBRegressor

from setup_notebook import setup_path
setup_path()
from src.model_utils import *
import joblib

## 2. Dataload & preprocessamento

In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_house_prices_v1.joblib')
preprocessador=temp['preprocessador']
colnull_train = temp['colnull_train']

# =====================================================
# 📁 1. Leitura dos dados & Separação das bases
# =====================================================
DATA_DIR = BASE / "data" / "processed"
X_train = pd.read_csv(DATA_DIR / "X_train_final.csv")
X_val  = pd.read_csv(DATA_DIR / "X_test_final.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_final.csv").squeeze()
y_val  = pd.read_csv(DATA_DIR / "y_test_final.csv").squeeze()

    
# =====================================================
# 📁 2. Modelo e pipeline
# =====================================================
model_xg2    = XGBRegressor( objective='reg:squarederror', n_estimators=700,
                         subsample= 0.6,reg_lambda= 0.5,reg_alpha= 1.0,
                         max_depth= 3,learning_rate= 0.073,
                         colsample_bytree= 0.7,
                         n_jobs=-1 ) 
XGB2         = pipe_models(model_xg2 , preprocessador)

In [3]:
# =====================================================
# 3.Treino & Teste
# =====================================================
XGB2.fit(X_train, y_train)
y_pred = XGB2.predict(X_val)
res2 = metricas_model(y_val, y_pred, 'XGBoost 2','on')

🤖 XGBOOST 2
MAE:   0.0838
RMSE:  0.124
R²:   0.9093


## 3. Kaggle

In [4]:
# =====================================================
# Submissão Kaggle
# =====================================================
# 1Leitura da base de teste oficial
base = pd.read_csv("/home/akel/PycharmProjects/Kaggle/HousePrices/data/raw/test.csv")

#  Remoção das colunas com muitos nulos (definidas no treino)
df_test = base.drop(colnull_train, axis=1, errors="ignore")

# Separação do ID
id_test = df_test["Id"]

# Features (sem Id)
X_test = df_test.drop(columns=["Id"])

# 🤖  Predição (pipeline completo: preprocessador + modelo)
y_pred_log = XGB2.predict(X_test)
y_pred = np.expm1(y_pred_log)


#  DataFrame de submissão
submission = pd.DataFrame({
    "Id": id_test,
    "SalePrice": y_pred
})

# 💾 Salvando arquivo CSV (formato Kaggle)
submission_path = ("/home/akel/PycharmProjects/Kaggle/HousePrices/data/submission_XGB2_tunned_v2.csv")

#submission.to_csv(submission_path, index=False)
print("✅ Arquivo de submissão salvo com sucesso!")

✅ Arquivo de submissão salvo com sucesso!
